# Tâche #2 : Classification d'incidents avec un réseau  récurrent et des *embeddings* Spacy

Cette tâche est similaire à la précédente et vous réutilisez les mêmes fichiers d’entraînement, de validation et de test. Cependant, vous devez utiliser des réseaux récurrents pour classifier les textes. Plus particulièrement, vous devez entraîner un réseau de neurones LSTM pour encoder les textes et une couche linéaire pour faire la classification des textes. 

Les consignes pour cette tâche sont: 
- 	Nom du notebook : rnn.ipynb
- 	Tokenisation : Utilisation de Spacy. 
- 	Plongements de mots : Ceux de Spacy. 
- 	Normalisation : Aucune normalisation. 
- 	Structure du réseau : Un réseau LSTM avec 1 seule couche pour l’encodage de textes. Je vous laisse déterminer la taille de cette couche (à expliquer). 
- 	Analyse : Comparer les résultats obtenus avec un réseau unidirectionnel et un réseau bidirectionnel. Si vous éprouvez des difficultés à entraîner les 2 réseaux dans un même notebook, faites une copie et nommez le 2e fichier rnn-bidirectionnel.ipynb.
- 	Expliquez comment les modèles sont utilisés pour faire la classification d’un texte. 
- 	Présentez clairement vos résultats et faites-en l’analyse. 


Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle. 

## 1. Création du jeu de données (*dataset*)

In [17]:
import spacy

spacy_model = spacy.load("en_core_web_md")
embedding_size = spacy_model.meta['vectors']['width']


import pandas as pd
import json
import numpy as np
# Assurez-vous que le modèle de langue de spacy est téléchargé
# python -m spacy download fr_core_news_md (par exemple pour le français)

# Charger le modèle de langue de spacy

# Définition des chemins vers les fichiers de données
train_data_path = './data/incidents_train.json'
dev_data_path = './data/incidents_dev.json'
test_data_path = './data/incidents_test.json'

def load_incident_dataset(filename):
    with open(filename, 'r') as fp:
        incident_list = json.load(fp)
        
        text = [item["text"] for item in incident_list]
        target = np.array([int(item["label"]) for item in incident_list])
         
    return text, target


# Créer les DataFrames pour chaque partition de données
train_list, train_target = load_incident_dataset(train_data_path)
dev_list, dev_target = load_incident_dataset(dev_data_path)
test_list, test_target = load_incident_dataset(test_data_path)

# Affichage de l'information de base sur les DataFrames
display(f"Train data: text_size {len(train_list)}, target_size {len(train_target)}")
display(f"Dev data: text_size {len(dev_list)}, target_size {len(dev_target)}")
display(f"Test data: text_size {len(test_list)}, target_size {len(test_target)}")



# Vérification des premiers enregistrements dans l'ensemble d'entraînement
train_list[0]


'Train data: text_size 2475, target_size 2475'

'Dev data: text_size 531, target_size 531'

'Test data: text_size 531, target_size 531'

' At approximately 8:50 a.m. on October 29  1997  Employee #1 was painting a  single story house at 2657 7th Ave  Sacramento  CA. He was caulking around the  peak of the roof line on the west side of the house  20 ft above the ground.  He was working off of a 24 ft aluminum extension ladder so that his feet were  approximately 12 to 13 feet above the ground. Employee #1 fell and suffered a  concussion and two dislocated discs in his lower back and was hospitalized.  The ladder was not secured to prevent movement.                                 '

In [18]:
import numpy as np

padding_token = "<PAD>"   # mot 0
unk_token = "<UNK>"    # mot 1
zero_vec_embedding = np.zeros(embedding_size, dtype=np.float64)

id2word = {}
id2word[0] = padding_token 
id2word[1] = unk_token 

word2id = {}
word2id[padding_token] = 0
word2id[unk_token] = 1

id2embedding = {}
id2embedding[0] = zero_vec_embedding
id2embedding[1] = zero_vec_embedding

## 2. Gestion de plongements de mots (*embeddings*)

In [12]:
from torch import FloatTensor

def get_spacy_embeddings(text, spacy_analyzer=spacy_model):
    doc = spacy_analyzer(text)
    embeddings = [token.vector for token in doc]
    return FloatTensor(embeddings)

Dimensions des embeddings de cet exemple: tensor([[  1.6849,   1.9826,  -0.7774,  ...,  -1.8289,  -2.9454,  -0.8334],
        [  1.4750,   6.0078,   1.1205,  ...,  -0.2289,  -0.8597,   9.7466],
        [ 11.4920,   2.9806,  15.9170,  ...,  12.5780, -12.9360,  -8.7743],
        [ -3.4108,   0.0728,   2.2763,  ...,  -2.1559,  -2.4203,  -1.7607]])


In [26]:
word_index = 2
vocab = word2id.keys()
for question in train_list:
    for word in spacy_model(question):
        if word.text not in vocab:
            word2id[word.text] = word_index
            id2word[word_index] = word.text
            id2embedding[word_index] = word.vector
            word_index += 1

In [25]:
id2word[4]

'approximately'

In [30]:
from torch.utils.data import Dataset, DataLoader
from torch import FloatTensor, LongTensor, unsqueeze
from typing import List
import numpy as np

class SpacyDataset(Dataset):
    def __init__(self, dataset: List[str] , target: np.array, wordId: dict, model=spacy_model):
        self.dataset = dataset
        self.doc_embeddings = [None for _ in range(len(dataset))]
        self.target = target
        self.model = model
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        if self.doc_embeddings[index] is None:
            self.doc_embeddings[index] = self.get_spacy_embeddings(self.dataset[index])  
        return self.doc_embeddings[index]
    
    def get_spacy_embeddings(self, text):
        doc = self.model(text)
        embeddings = [token.vector for token in doc]
        embeddings = FloatTensor(embeddings)
        return unsqueeze(embeddings, dim=0)

In [31]:
# On finalise la construction des 3 jeux de données et leurs dataloaders
train_dataset = SpacyDataset(train_list, train_target, word2id)
valid_dataset = SpacyDataset(dev_list, dev_target, word2id)
test_dataset = SpacyDataset(test_list, test_target, word2id)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [33]:
train_dataset[1]

tensor([[[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
         [  4.9588, -14.8870,  -5.2563,  ...,   1.2186,  10.2210,   1.0143],
         [ -2.7033,  -1.0999,  -3.8415,  ...,  -1.6602,  -2.0079,   2.2271],
         ...,
         [ -0.4947,   0.4467,  -1.3499,  ...,  -0.5042,  -3.3310,   1.0950],
         [ -0.0765,  -4.6896,  -4.0431,  ...,   1.3040,  -0.5270,  -1.3622],
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000]]])

## 3. Création de modèle(s)

In [35]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class LSTMClassifier(nn.Module):
    def __init__(self, embeddings, hidden_state_size, nb_classes) :
        super(LSTMClassifier, self).__init__()
        self.embedding_layer = nn.Embedding.from_pretrained(embeddings)
        self.embedding_size = embeddings.size()[1]
        self.rnn = nn.LSTM(self.embedding_size, hidden_state_size, 1, batch_first=True)
        self.classification_layer = nn.Linear(hidden_state_size, nb_classes)
    
    def forward(self, x, x_lengths):
        x = self.embedding_layer(x)
        packed_batch = pack_padded_sequence(x, x_lengths, batch_first=True, enforce_sorted=False)
        _, last_hidden_state = self.rnn(packed_batch)  # On utilise le hidden state de la dernière cellule
        x = last_hidden_state.squeeze()  # Le LSTM a une seule couche, on retire cette dimension
        x = self.classification_layer(x)
        return x

## 4. Fonctions utilitaires

Vous pouvez mettre ici toutes les fonctions qui seront utiles pour les sections suivantes.

## 5. Entraînement de modèle(s)

## 6. Évaluation et analyse de résultats